____
**Universidad Tecnológica Nacional, Buenos Aires**<br/>
**Ingeniería Industrial**<br/>
**Cátedra de Ciencia de Datos - Curso I5521 - Turno jueves noche**<br/>
**Clase_02:** EDA : Analisis Exploratorio de los Datos<br/>
**Elaborado por:** Nicolas Aguirre
____

In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import utils as ut
from scipy.stats import skew

warnings.filterwarnings("ignore")

# Dataset: Subtes de CABA - Molinetes 2017-2023

Fuente: https://data.buenosaires.gob.ar/dataset/subte-viajes-molinetes

Es una buena practica definir al inicio de nuestro script/notebook las rutas de los archivos que vamos a utilizar, como asi tambien las rutas donde vamos a guardar los resultados de nuestro analisis.

In [ ]:
# Definicion de la ruta base de los datos
base_path = "path/to/your/data/"
plot_path = "path/where/you/want/to/save/plots/"

Diccionario Auxiliar con las rutas de los archivos de molinetes


Dado que la forma en la que los archivos estan disponiubles cambia a partir de 2022, vamos a tener dos diccionarios
uno para los archivos `viejos` y otro para los `nuevos`


In [ ]:
# Archivos viejos: un solo archivo por año
molientes_paths_olds = {
    "2017": base_path + "molinetes-2017/molinetes.csv",
    "2018": base_path + "molinetes-2018/molinetes.csv",
    "2019": base_path + "molinetes-2019/molinetes.csv",
    #        "2020": base_path + "molinetes-2020/molinetes.csv",
    "2021": base_path + "molinetes-2021/molinetes.csv",
}

# Archivos nuevos: un directorio con varios archivos dentro por año
molientes_paths_news = {
    "2022": base_path + "molinetes-2022",
    "2023": base_path + "molinetes-2023",
}

Exploremos rapidamente las columnas y como estna conformados los datos para cada grupo de años. 

In [ ]:
# 2017-2021
df = pd.read_csv(molientes_paths_olds["2017"], delimiter=",", encoding="latin-1")
df.head()

In [ ]:
# 2022-2023
df = pd.read_csv(
    molientes_paths_news["2023"] + "/fixed_202306_PAX15min-ABC.csv",
    delimiter=";",
    engine="python",
    encoding="latin-1",
)
df.head()

Se puede obervar que a partir del año 2022, los nombres y tipos de datos de las columnas cambiaron.
El cientifico de datos debe estar atento a estos cambios, ya que pueden afectar el análisis y los resultados obtenidos.
En tal sentido la actividad de pre-procesamiento de los datos es fundamental en pos de consolidar un dataset coherente y consistente.

## Estaciones de Subte por Línea (ordenado)

In [ ]:
# Los nombres unificados de las estaciones por cada linea
orden_linea_a = ["Plaza de Mayo", "Peru", "Piedras", "Lima", "Saenz Peña", "Congreso", "Pasco", "Alberti", "Plaza Miserere", "Loria", "Castro Barros", "Rio de Janeiro", "Acoyte", "Primera Junta", "Puan", "Carabobo", "Flores", "San Pedrito",]
orden_linea_b = ["Leandro N. Alem", "Florida", "Carlos Pellegrini", "Uruguay", "Callao", "Pasteur", "Pueyrredon", "Carlos Gardel", "Medrano", "Angel Gallardo", "Malabia", "Dorrego", "Federico Lacroze", "Tronador", "Los Incas", "Echeverria", "Rosas",]
orden_linea_c = ["Constitucion", "San Juan", "Independencia", "Mariano Moreno", "Avenida de Mayo", "Diagonal Norte", "Lavalle", "General San Martin", "Retiro",]
orden_linea_d = ["Catedral", "9 de Julio", "Tribunales", "Callao", "Facultad de Medicina", "Pueyrredon", "Aguero", "Bulnes", "Scalabrini Ortiz", "Plaza Italia", "Palermo", "Ministro Carranza", "Olleros", "Jose Hernandez", "Juramento", "Congreso de Tucuman",]
orden_linea_e = ["RETIRO", "CORREO CENTRAL", "CATALINAS", "Bolivar", "GENERAL BELGRANO", "Independencia", "San Jose", "Entre Rios", "Pichincha", "Jujuy", "URQUIZA", "Boedo", "Avenida La Plata", "JOSE MARIA MORENO", "EMILIO MITRE", "MEDALLA MILAGROSA", "VARELA", "PZA. DE LOS VIRREYES",]
orden_linea_h = list(reversed([ "Hospitales", "PATRICIOS", "Caseros", "Inclan", "Humberto I", "Venezuela", "Once", "Corrientes", "Cordoba", "Santa Fe", "Las Heras", "Facultad de Derecho"]))

In [ ]:
# Diccionario auxiliar que contenga los nombres ordenados de las estaciones de cada linea
orden_linea = {
    "A": [x.upper() for x in orden_linea_a],
    "B": [x.upper() for x in orden_linea_b],
    "C": [x.upper() for x in orden_linea_c],
    "D": [x.upper() for x in orden_linea_d],
    "E": [x.upper() for x in orden_linea_e],
    "H": [x.upper() for x in orden_linea_h],
}
# Diccionario auxiliar que contenga los colores de cada linea
colors = {
    "A": "deepskyblue",
    "B": "red",
    "C": "blue",
    "D": "green",
    "E": "darkviolet",
    "H": "yellow",
}

In [ ]:
print(orden_linea["A"])

In [ ]:
print(colors["A"])

In [ ]:
# df = ut.get_all_data(molientes_path_old = molientes_paths_olds, molinetes_path_new = molientes_paths_news)
# Version simplificada sin los datos viejos
df = ut.get_all_data(molientes_path_old=None, molinetes_path_new=molientes_paths_news)

In [ ]:
# Las columnas PAX_PAGOS, PAX_PASES_PAGOS y PAX_FREQ no las utilizaremos, asi que vamos a eliminarlas para ahorrar memoria.
# Trabajaremos directamente con la columna TOTAL que es la suma de las tres anteriores.
df.drop(columns=["PAX_PAGOS", "PAX_PASES_PAGOS", "PAX_FREQ"], inplace=True)

---

=== NOTA: Como importar funciones auxiliares ===

---

Si observamos, al principio de la notebook hemos importado una libreria llamada `utils.py`, la cual contiene funciones auxiliares que nos permiten mantener el código/notebook más limpio y organizado.

En `utils.py` se declararon las funciones `get_all_data`, `load_subte_data_new`, `load_subte_data_old` y `clean_subte_dataframes`. 

Como se eligio el alias `ut` para importar `utils.py`, ahora podemos acceder a cualquiera de las funciones definidas en `utils.py` utilizando el prefijo `ut.` seguido del nombre de la función.

En particular, como `ut.get_all_data` ya incluye la carga y limpieza de los datos, podemos utilizarla directamente para obtener el DataFrame consolidado y limpio.

Veamos a continuación una breve descripción de las funciones definidas en `utils.py`:

```python
def get_all_data(molientes_path_old, molinetes_path_new):
    """
    Función que unifica la carga y limpieza de todos los datos del subte.
    """
    # === PROCESAMIENTO CONDICIONAL DE DATOS ===
    # Solo procesamos datos antiguos si están disponibles
    if molientes_path_old:             
        # Cargamos los datos antiguos usando la función específica
        df_old = load_subte_data_old(molientes_path_old)
        # Aplicamos el proceso de limpieza a los datos antiguos
        df_old = clean_subte_dataframes(df_old)
        
    # Solo procesamos datos nuevos si están disponibles
    if molinetes_path_new:
        # Cargamos los datos nuevos usando la función específica
        df_new = load_subte_data_new(molinetes_path_new)
        # Aplicamos el proceso de limpieza a los datos nuevos
        df_new = clean_subte_dataframes(df_new)
        
    # === CONSOLIDACIÓN FINAL ===
    # CODIGO OMITIDO (ver utils.py)

    return df
```

```python
def load_subte_data_new(molinetes_path_new):
    """
    Función para cargar y procesar datos nuevos de molinetes del subte.
    Esta función maneja archivos más recientes que pueden estar organizados en carpetas.    
    """
    # Diccionario para almacenar los DataFrames de cada año
    dict_of_dataframes_news = {}
    # Lista para almacenar información sobre las columnas (para debugging)
    list_of_colums = []
    # Columnas que queremos eliminar (incluye caracteres especiales por encoding)
    delet_cols = [ 'Â¿Fuera de rango?', 'Ã¹Fuera de rango?', 'DIA','HORA','TIPO_DIA']
    # Mapeo para renombrar columnas y estandarizar nombres
    rename_cols = [
    ('fecha', 'FECHA'), 
    ]

    # Iteramos sobre cada año y su carpeta correspondiente
    for year, folder in molinetes_path_new.items():
        # root: carpeta actual, dirs: subcarpetas, files: archivos en la carpeta actual
        for root, dirs, files in os.walk(folder):
            # Procesamos cada archivo en la carpeta actual
            for file in files:
                # Solo procesamos archivos CSV
                if file.endswith(".csv"):
                    current_df = pd.read_csv(os.path.join(root, file), delimiter=';', engine='python', encoding='latin-1')
                    # CODIGO OMITIDO (ver utils.py)
                # Concatenamos el DataFrame actual con el DataFrame acumulado del año
                df = pd.concat([df, current_df], ignore_index=True)
            # Guardamos el DataFrame consolidado del año
            dict_of_dataframes_news[year] = df
    # Concatenamos todos los años en un único DataFrame
    df_concat = pd.concat(dict_of_dataframes_news.values(), ignore_index=True)

    return df_concat
```

```python
def clean_subte_dataframes(df):
    """
    Función para limpiar y estandarizar los datos del subte.
    Realiza múltiples operaciones de limpieza en las columnas LINEA y ESTACION,
    maneja valores nulos y agrega columnas de tiempo útiles para el análisis.
    """
    
    # === LIMPIEZA DE LA COLUMNA LINEA ===
    df["LINEA"] = df["LINEA"].str.replace(r'Linea?', '', regex=True)
    df["LINEA"] = df["LINEA"].str.replace(r'LINEA_?', '', regex=True)
    
    # === LIMPIEZA DE LA COLUMNA ESTACION ===
    df["ESTACION"] = df["ESTACION"].str.upper()
    
    # Lista de diferentes formas en que aparece "AGÜERO" debido a problemas de encoding
    # Estos caracteres raros aparecen cuando hay problemas con acentos y caracteres especiales
    aguero_replace = ['AGÃ¼ERO', 'AGÃ\x83Â¼ERO' ,'AGÂ\x81ERO', 'AGÃ\x83Â\x83Ã\x82Â¼ERO' ,  'AGÏ¿½ERO', 'AGÃ\x82Â³ERO' ,  'AGÃ¯Â¿Â½ERO',  'AGÃ\x82Â\x81ERO']
    
    # Reemplazamos todas las variaciones problemáticas de "AGÜERO" con la forma estándar
    for a in aguero_replace:
        df["ESTACION"] = df["ESTACION"].str.replace(a, 'AGUERO')
    
    # Estandarizamos otros nombres de estaciones problemáticos
    # .*SAENZ.* captura cualquier variación que contenga "SAENZ"
    df["ESTACION"] = df["ESTACION"].replace(r'.*SAENZ .*', 'SAENZ PEÑA', regex=True)
    
    # Removemos sufijos específicos de algunas estaciones (probablemente códigos internos)
    df["ESTACION"] = df["ESTACION"].replace('CALLAO.B', 'CALLAO')
    # CODIGO OMITIDO (ver utils.py)
    
    # === ANÁLISIS DE VALORES NULOS ===
    
    # === ELIMINACIÓN DE FILAS CON VALORES NULOS ===

    # === CREACIÓN DE COLUMNAS TEMPORALES ===
    df['MONTH'] = df['FECHA'].dt.month      # Mes (1-12)
    df['YEAR'] = df['FECHA'].dt.year        # Año
    df['DAY'] = df['FECHA'].dt.weekday      # Día de la semana (0=Lunes, 6=Domingo)
    
    return df
```

Para mas detalles, ver y/o modificar `utils.py`.

---


In [ ]:
df.head()

In [ ]:
linea_estacion_df = df[["LINEA", "ESTACION"]].drop_duplicates()
linea_estacion_df

## Suma por ESTACION/LINEA

In [ ]:
# Agrupa el DataFrame por estación y línea, luego suma la columna 'TOTAL' para cada grupo
# Esto nos permite ver cuántos pasajeros totales pasaron por cada estación
sum_by_ESTACIONES = df.groupby(["ESTACION", "LINEA"])["TOTAL"].sum().reset_index()

# Crea una lista ordenada de estaciones según el total de pasajeros (de mayor a menor)
# Esta lista se utilizará más adelante para ordenar las estaciones en las visualizaciones
order_by_ESTACIONES = sum_by_ESTACIONES.sort_values("TOTAL", ascending=False)["ESTACION"].tolist()

# Calcula el porcentaje que representa cada estación sobre el total de pasajeros
# La función sum() sin parámetros suma todos los valores de la columna 'TOTAL'
sum_by_ESTACIONES['%TOTAL'] = sum_by_ESTACIONES['TOTAL'] / sum_by_ESTACIONES['TOTAL'].sum() * 100

# Redondea los porcentajes a 2 decimales para mejorar la legibilidad
sum_by_ESTACIONES["%TOTAL"] = sum_by_ESTACIONES["%TOTAL"].round(2)

# Save sum_by_ESTACIONES
sum_by_ESTACIONES.to_csv("sum_by_ESTACIONES.csv", index=False)

In [ ]:
plt.figure(figsize=(21, 7))
sns.barplot(
    data=sum_by_ESTACIONES,
    x="ESTACION",
    y="TOTAL",
    order=order_by_ESTACIONES,
    hue=sum_by_ESTACIONES["LINEA"],
    dodge=False,
    palette=colors,
)
plt.title("TOTAL PASAJEROS POR ESTACIONES (2017-2023)")
plt.xticks(rotation=90)
# guardemos la imagen en un archivo
plt.savefig("total_pasajeros_por_estacion.png")
plt.show()

# same plot but in % of the total
plt.figure(figsize=(21, 7))
sns.barplot(
    data=sum_by_ESTACIONES,
    x="ESTACION",
    y="%TOTAL",
    order=order_by_ESTACIONES,
    hue=sum_by_ESTACIONES["LINEA"],
    dodge=False,
    palette=colors,
)
plt.title("TOTAL PASAJEROS POR ESTACIONES (2017-2023)")
plt.xticks(rotation=90)
# guardemos la imagen en un archivo
plt.savefig("porcentaje_pasajeros_por_estacion.png")
plt.show()

In [ ]:
sum_by_LINEA = df.groupby("LINEA")["TOTAL"].sum().reset_index()
sum_by_LINEA["%TOTAL"] = sum_by_LINEA["TOTAL"] / sum_by_LINEA["TOTAL"].sum() * 100
# Para guardar el DataFrame en un archivo CSV ...
sum_by_LINEA.to_csv("sum_by_LINEA.csv", index=False)

In [ ]:
# plot bar chart
plt.figure(figsize=(12, 6))
sns.barplot(
    x="LINEA", y="TOTAL", data=sum_by_LINEA, hue=sum_by_LINEA["LINEA"], palette=colors
)
plt.title("TOTAL PASAJEROS POR LINEA (2017-2023)")
# save plot
plt.savefig("total_pasajeros_por_linea.png")
plt.show()
# plot by percentage
plt.figure(figsize=(12, 6))
sns.barplot(
    x="LINEA", y="%TOTAL", data=sum_by_LINEA, hue=sum_by_LINEA["LINEA"], palette=colors
)
plt.title("PORCENTAJE DE PASAJEROS POR LINEA (2017-2023)")
# save plot
plt.savefig("porcentaje_pasajeros_por_linea.png")
plt.show()

## Skewness

* Definicion

$$g_1=\frac{m_3}{m_2^{3/2}}$$

$$m_i=\frac{1}{N}\sum_{n=1}^N(x[n]-\bar{x})^i\$$

* Sin sesgo

$$G_1=\frac{k_3}{k_2^{3/2}}=\frac{\sqrt{N(N-1)}}{N-2}\frac{m_3}{m_2^{3/2}}$$


![Skewness Diagrams](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Negative_and_positive_skew_diagrams_%28English%29.svg/800px-Negative_and_positive_skew_diagrams_%28English%29.svg.png)

In [ ]:
# 1. Convertir la columna 'DESDE' a minutos desde la medianoche
df["DESDE_MINUTOS"] = pd.to_timedelta(df["DESDE"]).dt.total_seconds() / 60
df = df[(df["DESDE_MINUTOS"] >= 360) & (df["DESDE_MINUTOS"] <= 1439)]

In [ ]:
# Para cada línea única, recorrer cada estación
# y calcular skewness de DESDE_MINUTOS, guardarlos en una lista, para un dataframe final
skew_por_estacion = []
for linea in df["LINEA"].unique():
    print(f"Calculando skewness para {linea}")
    for estacion in df[df["LINEA"] == linea]["ESTACION"].unique():
        # obtener solo ESTACION y DESDE_MINUTOS del dataframe
        partial_df = df[(df["LINEA"] == linea) & (df["ESTACION"] == estacion)][
            ["ESTACION", "DESDE_MINUTOS", "DAY", "TOTAL"]
        ]
        # eliminar días de fin de semana (el simbolo ~ es para negar la condición)
        partial_df = partial_df[~partial_df["DAY"].isin([5, 6])]
        # repetir las filas de acuerdo al valor de 'TOTAL'
        partial_df = partial_df.loc[partial_df.index.repeat(partial_df["TOTAL"])]
        # calcular skewness de DESDE_MINUTOS
        skew_value = skew(partial_df["DESDE_MINUTOS"])
        skew_por_estacion.append([linea, estacion, skew_value])

# creamos un dataframe a partir de la lista
skew_df = pd.DataFrame(skew_por_estacion, columns=["LINEA", "ESTACION", "SKEW"])
# guardamos skew_df
skew_df.to_csv("skew_df.csv", index=False)

In [ ]:
# Veamos el dataframe skew_df
skew_df

In [ ]:
plt.figure(figsize=(21, 7))
sns.barplot(
    data=skew_df,
    x="ESTACION",
    y="SKEW",
    hue=skew_df["LINEA"],
    dodge=False,
    order=order_by_ESTACIONES,
    palette=colors,
)
# para agregar lineas horizontales en y=0.5 y y=-0.5 para tener de referencia
plt.axhline(y=0.5, color="k", linestyle="--")
plt.axhline(y=-0.5, color="k", linestyle="--")
plt.title("SKEWNESS PASAJEROS POR ESTACIONES (2017-2023)")
plt.xticks(rotation=90)
plt.show()


**Pregunta**: Que podemos decir de la estacion 

* `Constitucion`(C)?
* `Peru` (A)?
* `Once` (H)?


# Cantidad de pasajeros por estacion/horario/linea

Originalmente los datos viene en franjas de 15 minutos, pero para este análisis los vamos a agrupar en franjas de 45 minutos.


In [ ]:
# drop DESDE_MINUTOS column
df.drop("DESDE_MINUTOS", axis=1, inplace=True)

In [ ]:
# Definimos `delta` como el intervalo de tiempo en minutos para agrupar los datos
delta = 45

# Creamos una columna datetime combinando la fecha y la hora de inicio
# Esto nos permitirá manipular mejor los datos temporales
df["DESDE_DT"] = pd.to_datetime(df["FECHA"].astype(str) + " " + df["DESDE"])

# Nos aseguramos que el intervalo no sea menor a 15 minutos
# Esto es importante porque los datos originales ya vienen en franjas de 15 minutos
delta = max(15, delta)

# Redondeamos la hora hacia abajo al intervalo de delta minutos más cercano
# Por ejemplo, 8:37 con delta=45 se redondeará a 8:30
df["DESDE_DT"] = df["DESDE_DT"].dt.floor(f"{delta}T")

# Extraemos solo la parte de la hora (sin la fecha) y actualizamos la columna
# Esto nos permitirá agrupar por hora del día independientemente de la fecha
df["DESDE_DT"] = df["DESDE_DT"].dt.time

In [ ]:
df.head()

In [ ]:
for linea in df["LINEA"].unique():
    # Filtramos el DataFrame para obtener solo los datos de la línea actual
    df_linea = df[df["LINEA"] == linea]
    # Verificamos que todas las estaciones únicas para esta línea estén en la lista predefinida orden_linea
    # Esta verificación asegura que nuestros datos coincidan con el orden esperado de estaciones
    assert set(df_linea["ESTACION"].unique()) == set(orden_linea[linea])

    # Eliminamos sábados y domingos (donde DAY es 5 o 6)
    # Nos enfocamos solo en días laborables para analizar patrones habituales
    df_linea = df_linea[~df_linea["DAY"].isin([5, 6])]

    # Agrupamos los datos por estación y horario, sumando el total de pasajeros
    estaciones_hasta_df = df_linea.groupby(['ESTACION', 'DESDE_DT' ])['TOTAL'].sum().reset_index()

    # Eliminamos datos entre las 00:00 y las 05:30 (horario nocturno con poco tráfico)
    estaciones_hasta_df = estaciones_hasta_df[
        ~estaciones_hasta_df["DESDE_DT"].between(
            pd.to_datetime("00:00:00").time(), pd.to_datetime("05:30:00").time()
        )
    ]

    # Reducimos la escala dividiendo por 1 millón para mejor visualización
    estaciones_hasta_df["TOTAL"] = estaciones_hasta_df["TOTAL"] / 1e6

    # Creamos una figura de tamaño grande para mejor visualización
    plt.figure(figsize=(30, 7))

    # Generamos un gráfico de barras con seaborn
    # Usamos el orden predefinido de estaciones para mantener coherencia geográfica
    sns.barplot(
        data=estaciones_hasta_df,
        x="ESTACION",
        y="TOTAL",
        order=orden_linea[linea],
        hue=estaciones_hasta_df["DESDE_DT"],
        palette="magma_r",
    )

    # Agregamos título al gráfico
    plt.title(f"TOTAL PASAJEROS POR ESTACIONES DE LA LINEA {linea} Y SEGÚN HORARIO (2017-2023)", size=18)

    # Rotamos las etiquetas del eje X para mejor lectura
    plt.xticks(rotation=45)

    # Establecemos el límite del eje Y entre 0 y 4.5 millones
    plt.ylim(0, 4.5)

    # Etiquetamos los ejes
    plt.ylabel("TOTAL PASAJEROS (Millones)", size=18)
    plt.xlabel("ESTACIONES", size=18)

    # Colocamos la leyenda en la parte superior con 18 columnas para mostrar todos los horarios
    plt.legend(loc="upper center", ncol=18)

    # Guardamos el gráfico como archivo .png
    plt.savefig(f"horario_linea_{linea}.png", dpi=300, bbox_inches="tight")

    # Mostramos el gráfico
    plt.show()

# Discusión sobre los resultados

Ahora veamos lo mismo, pero en terminos de porcentajs relativos a cada linea...

In [ ]:
for linea in df["LINEA"].unique():
    # Filtramos el DataFrame para obtener solo los datos de la línea actual
    df_linea = df[df["LINEA"] == linea]

    # Eliminamos sábados y domingos (donde DAY es 5 o 6)
    df_linea = df_linea[~df_linea["DAY"].isin([5, 6])]

    # Agrupamos los datos por estación y horario, sumando el total de pasajeros
    estaciones_hasta_df = df_linea.groupby(["ESTACION", "DESDE_DT"])["TOTAL"].sum().reset_index()

    # Eliminamos datos entre las 00:00 y las 05:30 (horario nocturno con poco tráfico)
    estaciones_hasta_df = estaciones_hasta_df[
        ~estaciones_hasta_df["DESDE_DT"].between(
            pd.to_datetime("00:00:00").time(), pd.to_datetime("05:30:00").time()
        )
    ]

    # Calculamos el porcentaje de cada combinación estación-horario sobre el total de la línea
    estaciones_hasta_df["% TOTAL"] = estaciones_hasta_df["TOTAL"] / estaciones_hasta_df["TOTAL"].sum() * 100

    # Creamos una figura de tamaño grande para mejor visualización
    plt.figure(figsize=(30, 7))

    # Generamos un gráfico de barras con seaborn que muestra el porcentaje de pasajeros por estación
    # Usamos el orden predefinido de estaciones para mantener coherencia geográfica
    sns.barplot(
        data=estaciones_hasta_df,
        x="ESTACION",
        y="% TOTAL",
        order=orden_linea[linea],
        hue=estaciones_hasta_df["DESDE_DT"],
        palette="magma_r",
    )

    # Agregamos título al gráfico
    plt.title(f"% TOTAL DE PASAJEROS POR ESTACIONES DE LA LÍNEA {linea} Y SEGÚN HORARIO (2017-2023)", size=18)

    # Rotamos las etiquetas del eje X para mejor lectura
    plt.xticks(rotation=45)

    # Etiquetamos los ejes
    plt.ylabel("% TOTAL PASAJEROS", size=18)
    plt.xlabel("ESTACIONES", size=18)

    # Colocamos la leyenda en la parte superior con 18 columnas para mostrar todos los horarios
    plt.legend(loc="upper center", ncol=18)
    # Guardamos el gráfico como archivo .png
    plt.savefig(f"horario_linea_{linea}_percentage.png", dpi=300, bbox_inches="tight")
    plt.show()

## Discusión sobre los resultados